# Today's cryptocurrencies historical data on by Market Cap 

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import pendulum
from sqlalchemy import create_engine
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
from splinter import Browser
from bs4 import BeautifulSoup
import requests
from config import api_key, password
import urllib.request
from fs.osfs import OSFS

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\jich-\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


#### Context : Finance

#### Goal: Our goal is to create tables showing today’s 10 best/most profitable cryptocurrencies to invest in. We will look at historical data for the previous year and create a database by hourly and daily price changes. 

#### METHOD: we aim to do this by extracting data based on the ranks of the cryptocurrency and also the daily and hourly change of the cryptocurrency. 

#### FINDING DATA: Following are the sources for finding and collecting our data. Sources: Cryptocurrencies: https://coinmarketcap.com/ 
CSV files: https://www.cryptodatadownload.com/data/binance/
APIs: https://financialmodelingprep.com/api/v3/ https://rest.coinapi.io/v1/

#### Data collection tools: Splinter APIs BeautifulSoup

#### DATA AND CLEANUP: 
##### Cleanup process: Pandas SqlAlchemy Database: PostgreSQL

##### TEAM MEMBERS Juan Castaneda Elif Evrim Polat Nichole Edet

<div><h3 style="color:green;">Web Scraping - Obtaining today's best cryptocurrencies liste at https://coinmarketcap.com </h3></div>

In [3]:
# Obtaining Tables at the next link using pandas
url = 'https://coinmarketcap.com/'

In [4]:
tables = pd.read_html(url)

In [5]:
df = tables[0]
df.head()

,Unnamed: 0,#,Name,Price,24h %,7d %,Market Cap,Volume(24h),Circulating Supply,Last 7 Days,Unnamed: 10
0,NaN,1.0,Bitcoin1BTCBuy,"$63,073.03",0.08%,8.60%,"$1,178,452,848,812","$63,177,082,2681,001,650 BTC","18,683,943 BTC",NaN,NaN
1,NaN,2.0,Ethereum2ETHBuy,"$2,506.04",3.01%,20.38%,"$289,413,782,294","$31,857,934,43812,712,465 ETH","115,486,537 ETH",NaN,NaN
2,NaN,3.0,Binance Coin3BNBBuy,$531.11,1.49%,28.71%,"$82,074,306,200","$4,771,791,2558,984,519 BNB","154,532,785 BNB",NaN,NaN
3,NaN,4.0,XRP4XRP,$1.74,0.00%,68.51%,"$78,898,626,638","$14,584,570,3698,393,026,329 XRP","45,404,028,640 XRP",NaN,NaN
4,NaN,5.0,Tether5USDTBuy,$1.00,0.09%,0.00%,"$46,914,112,815","$152,320,344,625152,181,704,609 USDT","46,871,412,194 USDT",NaN,NaN


In [6]:
# using only the tables that will match our PostgreSQL database schema
columns = ['#', 'Name', 'Price']
cryptos_df = df[columns]
cryptos_df.head()

,#,Name,Price
0,1.0,Bitcoin1BTCBuy,"$63,073.03"
1,2.0,Ethereum2ETHBuy,"$2,506.04"
2,3.0,Binance Coin3BNBBuy,$531.11
3,4.0,XRP4XRP,$1.74
4,5.0,Tether5USDTBuy,$1.00


In [7]:
# converting # column values to integer
cryptos_df['#'] = cryptos_df['#'].astype(float).map("{:.0f}".format).copy()
cryptos_df.rename(columns={'#':'id','Name':'coin_name','Price':'latest_price'}, inplace=True)
# obtaiting only the first 10 coins
cryptos_df = cryptos_df[:10]

In [8]:
cryptos_df['latest_price'] = cryptos_df['latest_price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
cryptos_df.head()

,id,coin_name,latest_price
0,1,Bitcoin1BTCBuy,63073.03
1,2,Ethereum2ETHBuy,2506.04
2,3,Binance Coin3BNBBuy,531.11
3,4,XRP4XRP,1.74
4,5,Tether5USDTBuy,1.00


In [9]:
# Splitting Name column into 2 different columns which will contain the coin name and coin symbol using a number as a delimiter
# and using n=1 to do this only once, in case the coin name contains a number already
columnsplit = cryptos_df['coin_name'].str.split('(\d+)',n=1, expand=True)
columnsplit.head()

,0,1,2
0,Bitcoin,1,BTCBuy
1,Ethereum,2,ETHBuy
2,Binance Coin,3,BNBBuy
3,XRP,4,XRP
4,Tether,5,USDTBuy


###### Formatting table to match our postgreSQL table in our DB

In [10]:
columnsplit = cryptos_df['coin_name'].str.split('(\d+)',n=1, expand=True)
# since coin_symbol column originally contains a recommendation to "Buy" if applicable on coinmarketcap, we will get rid of that
# columnsplit[1] = columnsplit[1].str.replace('Buy', '', regex=True) by ignoring index 1
cryptos_df = cryptos_df.assign(coin_name=columnsplit[0],coin_symbol=columnsplit[2].str.replace('Buy',''))

In [11]:
cryptos_df

,id,coin_name,latest_price,coin_symbol
0,1,Bitcoin,63073.0300,BTC
1,2,Ethereum,2506.0400,ETH
2,3,Binance Coin,531.1100,BNB
3,4,XRP,1.7400,XRP
4,5,Tether,1.0000,USDT
5,6,Cardano,1.4600,ADA
6,7,Polkadot,43.1000,DOT
7,8,Dogecoin,0.2658,DOGE
8,9,Litecoin,295.9200,LTC
9,10,Uniswap,37.5300,UNI


In [12]:
# rearranging columns
columns = cryptos_df.columns.to_list()

In [13]:
# new order
columns = columns[:2] + [columns[-1]] + [columns[-2]]
columns

['id', 'coin_name', 'coin_symbol', 'latest_price']

In [14]:
cryptos_df = cryptos_df[columns]
cryptos_df

,id,coin_name,coin_symbol,latest_price
0,1,Bitcoin,BTC,63073.0300
1,2,Ethereum,ETH,2506.0400
2,3,Binance Coin,BNB,531.1100
3,4,XRP,XRP,1.7400
4,5,Tether,USDT,1.0000
5,6,Cardano,ADA,1.4600
6,7,Polkadot,DOT,43.1000
7,8,Dogecoin,DOGE,0.2658
8,9,Litecoin,LTC,295.9200
9,10,Uniswap,UNI,37.5300


<h3 style="color:orange;"> putting cryptos_df into coins table in pgadmin server</h3>

<h4 style="color:pink;"> Connecting to DB</h4>

In [15]:
connection_string = f"postgres:{password}@localhost:5433/cryptos_db"
engine = create_engine(f'postgresql://{connection_string}')

In [16]:
engine.table_names()

['coins', 'daily_Price', 'hourly_price']

In [17]:
cryptos_df.to_sql(name='coins', con=engine, if_exists='append', index=False)

In [18]:
# checking for changes in pgadmin cryoptos_db
pd.read_sql_query('select * from coins', con=engine)

,id,coin_name,coin_symbol,latest_price
0,1,Bitcoin,BTC,63073.0300
1,2,Ethereum,ETH,2506.0400
2,3,Binance Coin,BNB,531.1100
3,4,XRP,XRP,1.7400
4,5,Tether,USDT,1.0000
5,6,Cardano,ADA,1.4600
6,7,Polkadot,DOT,43.1000
7,8,Dogecoin,DOGE,0.2658
8,9,Litecoin,LTC,295.9200
9,10,Uniswap,UNI,37.5300


<div><h3 style="color:green;">Dowloading the CSV files if available at https://www.cryptodatadownload.com/data/binance/</h3></div>

In [19]:
# Obtaining symbols in the dataframe and storing in a list
symbols = cryptos_df['coin_symbol'].to_list()
symbols

['BTC', 'ETH', 'BNB', 'XRP', 'USDT', 'ADA', 'DOT', 'DOGE', 'LTC', 'UNI']

###### Web Scraping using beautiulsoup

In [20]:
# URL of page to be scraped
url = 'https://www.cryptodatadownload.com/data/binance/'
# instantiating the webdriver for Chrome!!!
browser.visit(url)
# Getting the webpage content
html = browser.html
# parsing our html plain text to a BS object
soup = BeautifulSoup(html, 'html.parser')
browser.quit()

In [21]:
prhs = soup.find_all('p')

In [22]:
bnb_cryptos = prhs[3]
links = bnb_cryptos.find_all('a')
complete_links = ["https://www.cryptodatadownload.com" + link['href'] for link in links if "cdd" in link['href']]

In [23]:
complete_links

['https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_minute.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_minute.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_minute.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_NEOUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_NEOUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_NEOUSDT_minute.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_minute.c

In [24]:
len(complete_links)

51

In [25]:
with OSFS("./Resources") as myfs:
    if(not myfs.exists("datasets")):
        myfs.makedir("datasets")
        print("Created datasets folder")

In [26]:
crypto_data_found = []
for symbol in (symbols):
    for index,link in enumerate(complete_links):
        if symbol in link:
            if symbol == "USDT":
                crypto_data_found.append((symbol,False))
            else:
                crypto_data_found.append((symbol,True))
            break
        if index==len(complete_links)-1:
            crypto_data_found.append((symbol,False))

In [27]:
#when true it means the file is on https://www.cryptodatadownload.com/cdd/Binance, otherwise we have to use the API
crypto_data_found

[('BTC', True),
 ('ETH', True),
 ('BNB', True),
 ('XRP', True),
 ('USDT', False),
 ('ADA', True),
 ('DOT', False),
 ('DOGE', False),
 ('LTC', True),
 ('UNI', False)]

###### coinAPI setup and use

In [28]:
def get_historic_data(symbol,time_start,time_end,period="1DAY",limit=10000):
    url = f"https://rest.coinapi.io/v1/ohlcv/{symbol}/USD/history?period_id={period}&limit={limit}&time_start={time_start}&time_end={time_end}"
    headers = {"X-CoinAPI-Key" : api_key}
    response = requests.get(url, headers = headers)

    if(response.status_code == 429):
        # API responses exhausted
        return "Too many requests."
    historical_data = response.json()
    print("getting data....")
    try:
        temp_df = pd.DataFrame(historical_data)
        if period == "1DAY":
            temp_df.to_csv(f"./Resources/datasets/{symbol}USDT_d.csv", index=False)
            print(f"{symbol} successfully exported")
        elif period == "1HRS":
            temp_df.to_csv(f"./Resources/datasets/{symbol}USDT_1h.csv", index=False)
            print(f"{symbol} successfully exported")
    except Exception as e:
        print(e)
    

###### Obaiting CSV files

In [29]:
for symbol,found in crypto_data_found:
    if found:
        try:
            file_name = f"{symbol}USDT_d.csv"
            daily_records = f"https://www.cryptodatadownload.com/cdd/Binance_{symbol}USDT_d.csv"
            urllib.request.urlretrieve(daily_records, f"./Resources/datasets/{file_name}")
            print(daily_records,"Downloaded")
            file_name = f"{symbol}USDT_1h.csv"
            hourly_records = f"https://www.cryptodatadownload.com/cdd/Binance_{symbol}USDT_1h.csv"
            file_name = f"{symbol}USDT_1h.csv"
            urllib.request.urlretrieve(hourly_records, f"./Resources/datasets/{file_name}")
            print(hourly_records,"Downloaded")
        except Exception as e:
            print(e)
    else:
        try:
            today = pendulum.now().format("YYYY-MM-DDTHH:mm:ss")
            year_ago= pendulum.now().subtract(years=1).format("YYYY-MM-DDTHH:mm:ss")
            get_historic_data(symbol,year_ago,today,period="1DAY")
            get_historic_data(symbol,year_ago,today,period="1HRS")
            print(f"Obtaining {symbol} historical data")
        except Exception as e:
            print(e)

https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_1h.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_1h.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_1h.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_XRPUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_XRPUSDT_1h.csv Downloaded
Obtaining USDT historical data
https://www.cryptodatadownload.com/cdd/Binance_ADAUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_ADAUSDT_1h.csv Downloaded
Obtaining DOT historical data
Obtaining DOGE historical data
https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_1h.csv Downloaded
Obtaining UNI historical data


### Cleaning up CSV files from https://www.cryptodatadownload.com/ EXAMPLE

In [30]:
btc_path = "./Resources/datasets/BTCUSDT_1h.csv"
# using skiprows to ignore first row in CSV file
df = pd.read_csv(btc_path, skiprows=1)

In [31]:
df

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount
0,1.618531e+12,2021-04-16 00:00:00,BTC/USDT,63158.74,63326.80,63158.73,63222.86,118.809924,7.515525e+06,4401.0
1,1.618528e+12,2021-04-15 23:00:00,BTC/USDT,63252.27,63339.99,62963.78,63159.98,1659.527690,1.048196e+08,61042.0
2,1.618524e+12,2021-04-15 22:00:00,BTC/USDT,63369.51,63518.22,63179.20,63252.27,2054.258481,1.301072e+08,77169.0
3,1.618520e+12,2021-04-15 21:00:00,BTC/USDT,63350.88,63800.00,63226.72,63369.50,2038.260233,1.293121e+08,91003.0
4,1.618517e+12,2021-04-15 20:00:00,BTC/USDT,63363.29,63592.24,63181.37,63350.89,3047.729738,1.932230e+08,119185.0
...,...,...,...,...,...,...,...,...,...,...
34742,1.502957e+09,2017-08-17 08-AM,BTC/USDT,4349.99,4377.85,4333.32,4360.69,0.949900,4.139700e+03,NaN
34743,1.502953e+09,2017-08-17 07-AM,BTC/USDT,4324.35,4349.99,4287.41,4349.99,4.440000,1.924106e+04,NaN
34744,1.502950e+09,2017-08-17 06-AM,BTC/USDT,4315.32,4345.45,4309.37,4324.35,7.230000,3.128231e+04,NaN
34745,1.502946e+09,2017-08-17 05-AM,BTC/USDT,4308.83,4328.69,4291.37,4315.32,23.230000,1.003048e+05,NaN


In [32]:
df['coin_id'] = 1
btc_df = df[['coin_id','date','open','high','low','close']]
btc_df

,coin_id,date,open,high,low,close
0,1,2021-04-16 00:00:00,63158.74,63326.80,63158.73,63222.86
1,1,2021-04-15 23:00:00,63252.27,63339.99,62963.78,63159.98
2,1,2021-04-15 22:00:00,63369.51,63518.22,63179.20,63252.27
3,1,2021-04-15 21:00:00,63350.88,63800.00,63226.72,63369.50
4,1,2021-04-15 20:00:00,63363.29,63592.24,63181.37,63350.89
...,...,...,...,...,...,...
34742,1,2017-08-17 08-AM,4349.99,4377.85,4333.32,4360.69
34743,1,2017-08-17 07-AM,4324.35,4349.99,4287.41,4349.99
34744,1,2017-08-17 06-AM,4315.32,4345.45,4309.37,4324.35
34745,1,2017-08-17 05-AM,4308.83,4328.69,4291.37,4315.32


### Cleaning up CSV files from https://www.coinapi.io/ EXAMPLE

In [33]:
dot_path = "./Resources/datasets/DOTUSDT_1h.csv"
df1 = pd.read_csv(dot_path)

In [34]:
df1

,time_period_start,time_period_end,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count
0,2020-08-18T17:00:00.0000000Z,2020-08-18T18:00:00.0000000Z,2020-08-18T17:02:07.9180000Z,2020-08-18T17:59:57.7310000Z,50.0000,50.7000,2.9000,2.9800,1.034595e+06,2325
1,2020-08-18T18:00:00.0000000Z,2020-08-18T19:00:00.0000000Z,2020-08-18T18:00:04.3460000Z,2020-08-18T18:59:50.3460000Z,2.9800,4.0000,2.9800,3.7000,7.682486e+05,1611
2,2020-08-18T19:00:00.0000000Z,2020-08-18T20:00:00.0000000Z,2020-08-18T19:00:00.0710000Z,2020-08-18T19:59:57.5690000Z,3.7000,3.7000,3.0000,3.1799,4.266289e+05,1530
3,2020-08-18T20:00:00.0000000Z,2020-08-18T21:00:00.0000000Z,2020-08-18T20:00:09.6050000Z,2020-08-18T20:59:53.7410000Z,3.1534,3.4500,3.0300,3.4425,6.609339e+05,1127
4,2020-08-18T21:00:00.0000000Z,2020-08-18T22:00:00.0000000Z,2020-08-18T21:00:12.5270000Z,2020-08-18T21:59:56.2230000Z,3.4425,3.4900,3.1500,3.2830,3.062926e+05,790
...,...,...,...,...,...,...,...,...,...,...
5720,2021-04-15T16:00:00.0000000Z,2021-04-15T17:00:00.0000000Z,2021-04-15T16:00:00.0010000Z,2021-04-15T16:59:57.4270000Z,42.9979,43.3478,42.5793,42.8700,2.979506e+04,691
5721,2021-04-15T17:00:00.0000000Z,2021-04-15T18:00:00.0000000Z,2021-04-15T17:00:00.0160000Z,2021-04-15T17:59:57.8260000Z,42.8699,42.9738,42.5600,42.9738,2.016249e+04,426
5722,2021-04-15T18:00:00.0000000Z,2021-04-15T19:00:00.0000000Z,2021-04-15T18:00:00.0030000Z,2021-04-15T18:59:31.5430000Z,42.9677,44.1362,42.9677,43.6643,4.289705e+04,1127
5723,2021-04-15T19:00:00.0000000Z,2021-04-15T20:00:00.0000000Z,2021-04-15T19:00:00.0010000Z,2021-04-15T19:59:53.7690000Z,43.6600,44.0614,43.3000,43.6181,5.316087e+04,965


In [35]:
df1['coin_id'] = 1
odt_df = df1[['coin_id','time_period_start','price_open','price_high','price_low','price_close']]
odt_df.rename(columns={"time_period_start":"date","price_open":"open","price_high":"high","price_low":"low",
                       "price_close":"low"}, inplace=True)
columnsplit = odt_df['date'].str.split(".",n=1, expand=True)
odt_df = odt_df.assign(date=columnsplit[0].str.replace("T"," "))
pd.to_datetime(odt_df['date'], infer_datetime_format=True)
odt_df

,coin_id,date,open,high,low,low
0,1,2020-08-18 17:00:00,50.0000,50.7000,2.9000,2.9800
1,1,2020-08-18 18:00:00,2.9800,4.0000,2.9800,3.7000
2,1,2020-08-18 19:00:00,3.7000,3.7000,3.0000,3.1799
3,1,2020-08-18 20:00:00,3.1534,3.4500,3.0300,3.4425
4,1,2020-08-18 21:00:00,3.4425,3.4900,3.1500,3.2830
...,...,...,...,...,...,...
5720,1,2021-04-15 16:00:00,42.9979,43.3478,42.5793,42.8700
5721,1,2021-04-15 17:00:00,42.8699,42.9738,42.5600,42.9738
5722,1,2021-04-15 18:00:00,42.9677,44.1362,42.9677,43.6643
5723,1,2021-04-15 19:00:00,43.6600,44.0614,43.3000,43.6181


<div style="background-color:pink;font-size:20px;"><h3 style="color:red;"> We're going to utilize a for loop to automate the clean up process</h3></div>

In [55]:
for crypto,found in crypto_data_found:
    coin_id = cryptos_df.loc[cryptos_df['coin_symbol'].isin([crypto])]['id']
#     print(int(coin_id))
    if found:    # if found= True, it means the data was obtained from  https://www.cryptodatadownload.com/ 
        for i in ['1h','d']:
            df_path = f"./Resources/datasets/{crypto}USDT_{i}.csv"
            # using skiprows to ignore first row in CSV file
            df = pd.read_csv(df_path, skiprows=1)
            df['coin_id'] = int(coin_id)
            df_new = df[['coin_id','date','open','high','low','close']]
            print(df_new)
    if found == False: # the data was obtained from the coinAPI
        print(int(coin_id),crypto, "not found")
        for i in ['1h','d']:
            print(crypto)
            df_path = f"./Resources/datasets/{crypto}USDT_{i}.csv"
#             print(df_path)
            df = pd.read_csv(df_path) 
            df['coin_id'] = int(coin_id)
            df_new = df[['coin_id','time_period_start','price_open','price_high','price_low','price_close']]
            df_new.rename(columns={"time_period_start":"date","price_open":"open","price_high":"high","price_low":"low",
                                   "price_close":"low"}, inplace=True)
            columnsplit = df_new['date'].str.split(".",n=1, expand=True)
            df_new = df_new.assign(date=columnsplit[0].str.replace("T"," "))
            pd.to_datetime(df_new['date'], infer_datetime_format=True)
            print(df_new)
         

       coin_id                 date      open      high       low     close
0            1  2021-04-16 00:00:00  63158.74  63326.80  63158.73  63222.86
1            1  2021-04-15 23:00:00  63252.27  63339.99  62963.78  63159.98
2            1  2021-04-15 22:00:00  63369.51  63518.22  63179.20  63252.27
3            1  2021-04-15 21:00:00  63350.88  63800.00  63226.72  63369.50
4            1  2021-04-15 20:00:00  63363.29  63592.24  63181.37  63350.89
...        ...                  ...       ...       ...       ...       ...
34742        1     2017-08-17 08-AM   4349.99   4377.85   4333.32   4360.69
34743        1     2017-08-17 07-AM   4324.35   4349.99   4287.41   4349.99
34744        1     2017-08-17 06-AM   4315.32   4345.45   4309.37   4324.35
34745        1     2017-08-17 05-AM   4308.83   4328.69   4291.37   4315.32
34746        1     2017-08-17 04-AM  16199.91  16199.91   4261.32   4308.83

[34747 rows x 6 columns]
      coin_id                 date      open      high       l

      coin_id                 date     open     high      low      low
0           7  2020-08-18 17:00:00  50.0000  50.7000   2.9000   2.9800
1           7  2020-08-18 18:00:00   2.9800   4.0000   2.9800   3.7000
2           7  2020-08-18 19:00:00   3.7000   3.7000   3.0000   3.1799
3           7  2020-08-18 20:00:00   3.1534   3.4500   3.0300   3.4425
4           7  2020-08-18 21:00:00   3.4425   3.4900   3.1500   3.2830
...       ...                  ...      ...      ...      ...      ...
5720        7  2021-04-15 16:00:00  42.9979  43.3478  42.5793  42.8700
5721        7  2021-04-15 17:00:00  42.8699  42.9738  42.5600  42.9738
5722        7  2021-04-15 18:00:00  42.9677  44.1362  42.9677  43.6643
5723        7  2021-04-15 19:00:00  43.6600  44.0614  43.3000  43.6181
5724        7  2021-04-15 20:00:00  43.6180  43.9000  43.4474  43.5096

[5725 rows x 6 columns]
DOT
     coin_id                 date     open     high      low      low
0          7  2020-08-18 00:00:00  50.0000  50.70